In [67]:
import pandas
import re
import math
dRaw = pandas.read_csv("./KA_DATA/KA DEMO_OUTPUT (REVISION) - DEMO_OUTPUT.csv")

outputCSVHeaders = ['AGE', 'ALGOTYPE', 'DEMO', 'OVERALL.RESPONSE', 'KNOWERLEV','GENDER', 'ID','PASS_SORT','PASS_DOUBLE','PASS_HITCH','PASS_SORT_1','PASS_DOUBLE_1','PASS_HITCH_1', 'OVERALL.RESPONSE.NO', 'RESPONSE.NO', 'ACTION.NO', 'BUCKET', 'RESPONSE']

In [68]:
dRaw.head()

,ID,Tester,TestDate,AGE,GENDER,SORT_DEMO,SORT,DOUBLE_DEMO,DOUBLE,HITCH_DEMO,HITCH,SUCCESSOR_SORT,SUCCESSOR_DOUBLE,SUCCESSOR_HITCH,SUCCESSOR_SORT.1,SUCCESSOR_DOUBLE.1,SUCCESSOR_HITCH.1,KNOWER_LEV
0,1,NaN,NaN,3.737,FEMALE,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,2.0
1,2,NaN,NaN,3.515,MALE,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,3.0
2,3,NaN,NaN,3.625,MALE,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,3.0
3,4,NaN,NaN,4.821,FEMALE,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G","G/, /RR, G/, /RR, G/, /RR, G/, /RR","G/, /RR, G/, /RR, G/, /RR, G/, /RR","/RB, GB/, /RB, GB/, /RB, GB/, /RB, GB/","/RB, GB/, /RB, GB/, /RB, GB/, /RB, GB/",1.0,1.0,1.0,1.0,1.0,1.0,5.0
4,5,NaN,NaN,4.063,FEMALE,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,2.0


In [69]:
d = dRaw.loc[ (dRaw['SORT'].notnull()) | (dRaw['DOUBLE'].notnull()) | (dRaw['HITCH'].notnull()) ]

In [70]:
len(d), len(dRaw)

(33, 37)

In [71]:
d = d.loc[ (d['SORT']!="0") & (d['DOUBLE']!="0") & (d['HITCH']!="0") ]

In [72]:
d.tail()

,ID,Tester,TestDate,AGE,GENDER,SORT_DEMO,SORT,DOUBLE_DEMO,DOUBLE,HITCH_DEMO,HITCH,SUCCESSOR_SORT,SUCCESSOR_DOUBLE,SUCCESSOR_HITCH,SUCCESSOR_SORT.1,SUCCESSOR_DOUBLE.1,SUCCESSOR_HITCH.1,KNOWER_LEV
29,38,NaN,NaN,NaN,MALE,"G/, /R, G/, /R, G/, /R, G/, /R","R/, /G, R/, /G, R/, /G, R/, /G, R/, /G, R/, /G","R/, /GG, R/, /GG, R/, /GG, R/, /GG","GG/, /R, GG/, /R, GG/, /R, /R, G/, G/, /R, G/","RB/, /GB, RB/, /GB, RB/, /GB, RB/, /GB","RB/, /GB, RB/, /GB, RB/, /GB, RB/, RBRB/, RBRB...",1.0,1.0,1.0,1.0,1.0,1.0,10.0
33,42,NaN,NaN,NaN,FEMALE,"G/, /R, G/, /R, G/, /R, G/, /R","R/, /G, R/, /G, RR/, /G, R/, /G, /G, R/, /GG","R/, /GG, R/, /GG, R/, /GG, R/, /GG","R/, /G, R/, RR/, /GG, R/, /GG","RB/, /GB, RB/, /GB, RB/, /GB, RB/, /GB","RB/, /GB, RB/, /GB, RB/, /GB",1.0,1.0,1.0,1.0,0.0,1.0,2.0
34,43,NaN,NaN,NaN,FEMALE,"G/, /R, G/, /R, G/, /R, G/, /R","/G, R/, /G, R/, /G, /Rc, R/, GGc/, /GG, R/","R/, /GG, R/, /GG, R/, /GG, R/, /GG","R/, /GG, R/, /GG, R/, /GG, Rb/, R/, /GG, R/, /GG","RB/, /GB, RB/, /GB, RB/, /GB, RB/, /GB","RB/, /GB, RB/, /GBBb, /GB, RB/, /GB, RB/, /GB",1.0,1.0,1.0,1.0,1.0,1.0,6.0
35,45,NaN,NaN,NaN,MALE,"G/, /R, G/, /R, G/, /R, G/, /R","R/, /G, R/, /G, R/, /G, R/, /G, R/, /G","R/, /GG, R/, /GG, R/, /GG, R/, /GG","R/, /G, RR/, RRGc/ , /G, R/, R/, R/, /G, /G, /...","RB/, /GB, RB/, /GB, RB/, /GB, RB/, /GB","RB/, /GB, RB/, /GB, RB/, /GB",1.0,1.0,1.0,1.0,0.0,1.0,4.0
36,46,NaN,NaN,NaN,MALE,"G/, /R, G/, /R, G/, /R, G/, /R",NaN,"R/, /GG, R/, /GG, R/, /GG, R/, /GG","R/, RR/, /Rc, R/, R/, /G, /G","RB/, /GB, RB/, /GB, RB/, /GB, RB/, /GB","B/, G/, /R, /G, RR/",0.0,1.0,0.0,0.0,0.0,0.0,4.0


In [73]:
def regexParse (row, algoType):
    algoLabel = algoType
    demoLabel = algoType + "_DEMO"
    resString = row[algoLabel]
    dataStr = []
    overall_response_count = 0
    if pandas.isna(resString) : resString = " / "
    for ri, response in enumerate(re.split(r"\s*,\s*", resString)): # split on commas
        lb, rb = re.split(r"/", response) # split left and right buckets

        # remove whitespace
        lb = re.sub(r"\s", "", lb) 
        rb = re.sub(r"\s", "", rb)

        ##############################
        ## Process annotations
        ##############################

        # Remove xes
        lb = re.sub(r"x", "", lb)
        rb = re.sub(r"x", "", rb)

        # b -- put back into original container 
        # for now, we'll remove these
        # sometimes cm may occur if they correct and then put it back
#         lb = re.sub(r"[A-Z][cm]*b", "", lb)
#         rb = re.sub(r"[A-Z][cm]*b", "", rb)
#         or just remove all the b
        lb = re.sub(r"b", "", lb)
        rb = re.sub(r"b", "", rb)

        # add in corrections -- any characters followed by c, put into other bucket
        for x in re.findall("([A-Z])c", lb):
            rb += x
        for x in re.findall("([A-Z])c", rb):
            lb += x
        # now remove these from the original side
        lb = re.sub(r"[A-Z]c", "", lb)
        rb = re.sub(r"[A-Z]c", "", rb)

        # toss out m annotation for now
        lb = re.sub(r"m", "", lb)
        rb = re.sub(r"m", "", rb)

#         print rb + "," + lb
        assert not re.search(r"[^A-Z]", lb)
        assert not re.search(r"[^A-Z]", rb)

        for ai, a in enumerate(lb):
            singleAct = [row["AGE"], algoType, row[demoLabel], row[algoLabel], row['KNOWER_LEV'], row['GENDER'], row['ID'],row['SUCCESSOR_SORT'],row['SUCCESSOR_DOUBLE'],row['SUCCESSOR_HITCH'], row['SUCCESSOR_SORT.1'],row['SUCCESSOR_DOUBLE.1'],row['SUCCESSOR_HITCH.1'], overall_response_count, ri, ai, "left", a] 
            dataStr.append(singleAct)
            overall_response_count += 1

        for ai, a in enumerate(rb):
            singleAct = [row["AGE"], algoType, row[demoLabel], row[algoLabel],row['KNOWER_LEV'], row['GENDER'], row['ID'],row['SUCCESSOR_SORT'],row['SUCCESSOR_DOUBLE'],row['SUCCESSOR_HITCH'],row['SUCCESSOR_SORT.1'],row['SUCCESSOR_DOUBLE.1'],row['SUCCESSOR_HITCH.1'],overall_response_count, ri, ai, "right", a] 
            dataStr.append(singleAct)
            overall_response_count += 1
    return dataStr

In [74]:
def parseAll (d):
    dataStr = []
    for subjecti, row in d.iterrows(): 
        dataStr += regexParse(row, "SORT") + regexParse(row, "DOUBLE") + regexParse(row, "HITCH")
    return dataStr
            

In [77]:
dataStr = parseAll(d)
print dataStr[0:10]
finalDF = pandas.DataFrame(dataStr, columns=outputCSVHeaders)

[[4.821000000000001, 'SORT', 'R/, /G, R/, /G, R/, /G, R/, /G ', 'R/, /G, /RGxc, /G, R/, /G', 5.0, 'FEMALE', 4, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0, 0, 0, 'left', 'R'], [4.821000000000001, 'SORT', 'R/, /G, R/, /G, R/, /G, R/, /G ', 'R/, /G, /RGxc, /G, R/, /G', 5.0, 'FEMALE', 4, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1, 1, 0, 'right', 'G'], [4.821000000000001, 'SORT', 'R/, /G, R/, /G, R/, /G, R/, /G ', 'R/, /G, /RGxc, /G, R/, /G', 5.0, 'FEMALE', 4, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2, 2, 0, 'left', 'G'], [4.821000000000001, 'SORT', 'R/, /G, R/, /G, R/, /G, R/, /G ', 'R/, /G, /RGxc, /G, R/, /G', 5.0, 'FEMALE', 4, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3, 2, 0, 'right', 'R'], [4.821000000000001, 'SORT', 'R/, /G, R/, /G, R/, /G, R/, /G ', 'R/, /G, /RGxc, /G, R/, /G', 5.0, 'FEMALE', 4, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4, 3, 0, 'right', 'G'], [4.821000000000001, 'SORT', 'R/, /G, R/, /G, R/, /G, R/, /G ', 'R/, /G, /RGxc, /G, R/, /G', 5.0, 'FEMALE', 4, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 5, 4, 0, 'left', 'R'], [4.821000000000001

In [79]:
finalDF.head()

,AGE,ALGOTYPE,DEMO,OVERALL.RESPONSE,KNOWERLEV,GENDER,ID,PASS_SORT,PASS_DOUBLE,PASS_HITCH,PASS_SORT_1,PASS_DOUBLE_1,PASS_HITCH_1,OVERALL.RESPONSE.NO,RESPONSE.NO,ACTION.NO,BUCKET,RESPONSE
0,4.821,SORT,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G",5.0,FEMALE,4,1.0,1.0,1.0,1.0,1.0,1.0,0,0,0,left,R
1,4.821,SORT,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G",5.0,FEMALE,4,1.0,1.0,1.0,1.0,1.0,1.0,1,1,0,right,G
2,4.821,SORT,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G",5.0,FEMALE,4,1.0,1.0,1.0,1.0,1.0,1.0,2,2,0,left,G
3,4.821,SORT,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G",5.0,FEMALE,4,1.0,1.0,1.0,1.0,1.0,1.0,3,2,0,right,R
4,4.821,SORT,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G",5.0,FEMALE,4,1.0,1.0,1.0,1.0,1.0,1.0,4,3,0,right,G


In [80]:
newDF = finalDF.drop(columns=['ALGOTYPE', 'DEMO', 'OVERALL.RESPONSE', 'OVERALL.RESPONSE.NO', 'RESPONSE.NO', 'ACTION.NO', 'BUCKET', 'RESPONSE'])

In [83]:
newDF = newDF.drop_duplicates()
newDF.head()
# newDF.to_csv("./experimentData_CodingKL.csv")

In [87]:
finalDF['KL_AGE'] = finalDF['KNOWERLEV'].map(str) + "_" + finalDF['AGE'].map(str)
finalDF.head()

,AGE,ALGOTYPE,DEMO,OVERALL.RESPONSE,KNOWERLEV,GENDER,ID,PASS_SORT,PASS_DOUBLE,PASS_HITCH,PASS_SORT_1,PASS_DOUBLE_1,PASS_HITCH_1,OVERALL.RESPONSE.NO,RESPONSE.NO,ACTION.NO,BUCKET,RESPONSE,KL_AGE
0,4.821,SORT,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G",5.0,FEMALE,4,1.0,1.0,1.0,1.0,1.0,1.0,0,0,0,left,R,5.0_4.821
1,4.821,SORT,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G",5.0,FEMALE,4,1.0,1.0,1.0,1.0,1.0,1.0,1,1,0,right,G,5.0_4.821
2,4.821,SORT,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G",5.0,FEMALE,4,1.0,1.0,1.0,1.0,1.0,1.0,2,2,0,left,G,5.0_4.821
3,4.821,SORT,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G",5.0,FEMALE,4,1.0,1.0,1.0,1.0,1.0,1.0,3,2,0,right,R,5.0_4.821
4,4.821,SORT,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G",5.0,FEMALE,4,1.0,1.0,1.0,1.0,1.0,1.0,4,3,0,right,G,5.0_4.821


In [88]:
finalDF.to_csv("./experimentData_BallCodingKL.csv")